In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/

/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs


/Users/karolinegriesbach/.local/share/virtualenvs/consumption-based-costs-wGvc2ut4/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import pandas as pd

from src.utils.cost_handling import return_cost_per_run

# Categorize Costs
- Use grouping to categorize costs

In [3]:
from_date = "2024-01-01"
to_date="2025-06-23"
timestamp = "2025-06-23 13:23:56.520433"
path_to_dir_retraining = f"/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/{from_date}_to_{to_date}/targeting_and_retraining/"
path_to_retraining_data = f"{path_to_dir_retraining}map_nodes_to_prefect_runs_{from_date}_{to_date}.csv"
path_to_save = f"/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/{from_date}_to_{to_date}/costs/"
file_costs = f"final_retraining_costs_{from_date}_{to_date}.csv"

In [4]:
os.makedirs(path_to_save, exist_ok=True)

# Load Data

In [5]:
# loading targeting costs mapping (generated via analysis/3_cost_targeting.ipynb)
df_retraining = pd.read_csv(path_to_retraining_data)
df_retraining = df_retraining[[col for col in df_retraining.columns if "Unnamed" not in col]]
df_retraining.shape

(11088, 58)

In [6]:
# get cost per run
df_retraining = return_cost_per_run(df_retraining)
df_retraining = df_retraining.sort_values(by=["date","serviceName"])
df_retraining = df_retraining[["date","cloud","Prefect_Deployments","serviceName","tenant","audience","duration","sum_duration_serviceName","total_charge_of_serviceName","part_of_costs_per_run","cost_per_run"]]

In [7]:
vc = df_retraining.groupby(by=["date","serviceName"])["part_of_costs_per_run"].sum()
vc_all = vc.reset_index()
vc = vc_all[vc_all["part_of_costs_per_run"].round()!=1]
vc = vc[vc["serviceName"]!="General Purpose Server-g1.3-EU01"] # is used for agentpools as well, so can be != 1
if len(vc) > 0:
    raise ValueError(f"Some the costs does not match the sum: {vc}")

In [8]:
df_retraining = df_retraining[df_retraining["Prefect_Deployments"]=="retraining"]
df_retraining = df_retraining.reset_index(drop=True)
df_retraining

,date,cloud,Prefect_Deployments,serviceName,tenant,audience,duration,sum_duration_serviceName,total_charge_of_serviceName,part_of_costs_per_run,cost_per_run
0,2025-02-10,stackit,retraining,Compute Optimized Server-c1.3-EU01,pendix,618bc646fb99125ac1c4f593,61.59711,18877.987376,4.550782,0.003263,0.014849
1,2025-02-10,stackit,retraining,Memory Optimized Server-b1.4-EU01,rosental,671e21ff2c5ade4568f2daa9,962.21980,25965.819700,40.380014,0.037057,1.496369
2,2025-02-10,stackit,retraining,Memory Optimized Server-m1.3-EU01,lillydoo,NaN,962.37620,50315.866166,6.156801,0.019127,0.117759
3,2025-02-11,stackit,retraining,Memory Optimized Server-b1.4-EU01,rosental,65f05c6e1bb884124e0f910d,12950.28300,35326.221360,11.306404,0.366591,4.144829
4,2025-02-11,stackit,retraining,Memory Optimized Server-b1.4-EU01,rosental,65f05c6e1bb884124e0f910d,19248.40600,35326.221360,11.306404,0.544876,6.160587
...,...,...,...,...,...,...,...,...,...,...,...
1076,2025-06-19,stackit,retraining,Memory Optimized Server-m1.3-EU01,ective,67da6bb1ba5842f963fc5eb3,348.93698,60625.741956,9.042801,0.005756,0.052047
1077,2025-06-21,stackit,retraining,Memory Optimized Server-b1.4-EU01,rosental,678a5c81f3abb5123264a6a8,667.36334,667.363340,2.422801,1.000000,2.422801
1078,2025-06-23,NaN,retraining,NaN,toteach,NaN,74.74597,NaN,NaN,NaN,NaN
1079,2025-06-23,NaN,retraining,NaN,toteach,NaN,411.53630,NaN,NaN,NaN,NaN


In [9]:
df_retraining.to_csv(f"{path_to_save}{file_costs}",index=False)

# Cost: Server Runs per Day and serviceName

In [10]:
df_retraining.groupby(by=["date","serviceName","tenant","audience_id",""])

KeyError: 'audience_id'